### Import libraries

In [1]:
import sys
import os
import gc
import numpy as np
import pandas as pd 
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [2]:
home = os.path.expanduser("~")
sys.path.append(f"{home}/Documents/projects/CarPriceRegression/Machine_Learning/scripts/")

In [29]:
from util import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
set_gpu_limit(6)

### Import data for regular NN without categorical embedding

In [5]:
data = pd.read_csv(f"{home}/Documents/projects/CarPriceRegression/Processed_Data/car_onehot_data.csv")
features, y = data.drop("price",axis=1), data.price
X_train, X_test, y_train, y_test = train_test_split(features,y,test_size=0.1,random_state=2020)
X_train,X_dev,y_train,y_dev = train_test_split(X_train,y_train,test_size=0.11,random_state=2020)

### Garbage collect

In [6]:
gc.collect()
%reset -f out

Flushing output cache (0 entries)


### Set up base parameter for a base regression model

In [7]:
BATCH_SIZE = 256 
input_size = features.shape[1]
MAX_EPOCH = 100
lr = 1e-3
l2 = 0 
SIZES = [512,512]
METRICS = ["MAPE","MAE"]

### Set up a few common parameter and make tensorflow dataset

* notice: this dataset has pretty similar scale so no reason to normalize to help with gradient descent

In [8]:
train, dev = make_tensor_dataset(X_train,y_train,256), make_tensor_dataset(X_dev,y_dev,256)

### Set up base parameter for a base regression model

In [9]:
base_model = make_model(SIZES,input_size,METRICS,l2,lr)

In [10]:
base_model.summary()

Model: "NN_regressor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               105984    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 369,153
Trainable params: 369,153
Non-trainable params: 0
_________________________________________________________________


### Define early stop

In [11]:
earlystop = tfk.callbacks.EarlyStopping(monitor="val_MAPE",mode="min",
                                        patience=10,verbose=1,restore_best_weights=True)

In [ ]:
# history = base_model.fit(train, epochs=MAX_EPOCH,shuffle=True,
#                          verbose = 1, validation_data=dev,
#                          callbacks=[earlystop])

In [ ]:
# base_model.save("base_nn_regression_mdl.h5")

In [12]:
base_model = tfk.models.load_model("base_nn_regression_mdl.h5",custom_objects={"leaky_relu":tf.nn.leaky_relu})

In [ ]:
plot_metrics(history,"loss")

In [ ]:
plot_metrics(history,"mean_absolute_percentage_error")

### Check base model result

In [13]:
base_model.evaluate(X_test.values,y_test.values,batch_size=BATCH_SIZE)

39/39 [==============================] - 0s 1ms/step - loss: 7249358.5000 - mean_absolute_percentage_error: 10.8011 - mean_absolute_error: 1670.8715


[7249358.5, 10.801079750061035, 1670.8714599609375]

In [24]:
train_pred = base_model.predict(X_train.values,batch_size=256).reshape(-1)
test_pred = base_model.predict(X_test.values,batch_size=256).reshape(-1)

In [32]:
regression_metrics(base_model,X_train,y_train,X_test,y_test,True,train_pred,test_pred)

,r2_score,rmse,price_diff_abs_max
train,0.926519,2673.839310,1058.111443
test,0.922090,2692.463356,966.627930


#### Definitely can benefit from tuning. 